In [1]:
from pymongo import MongoClient
from pprint import pprint

In [2]:
client = MongoClient('localhost', 27017)
pipeline = [
    {
        "$project": {"_id": 1, "links": 1, "old": 1, "new": 1}
    }
]

In [3]:
data = list(client['enigma']['test'].aggregate(pipeline=pipeline))
pprint(data)

[{'_id': 24752893,
  'links': ['6786765789.0', '76675567'],
  'new': 1.1290058538953526e-05,
  'old': 0.0},
 {'_id': 6786765789.0,
  'links': ['24752893'],
  'new': 5.645029269476763e-06,
  'old': 0.0},
 {'_id': 76675567,
  'links': ['24752893', '6786765789.0'],
  'new': 1.1290058538953526e-05,
  'old': 0.0}]


In [4]:
for doc in data:
    doc['old'] = doc['new']
pprint(data)

[{'_id': 24752893,
  'links': ['6786765789.0', '76675567'],
  'new': 1.1290058538953526e-05,
  'old': 1.1290058538953526e-05},
 {'_id': 6786765789.0,
  'links': ['24752893'],
  'new': 5.645029269476763e-06,
  'old': 5.645029269476763e-06},
 {'_id': 76675567,
  'links': ['24752893', '6786765789.0'],
  'new': 1.1290058538953526e-05,
  'old': 1.1290058538953526e-05}]


In [5]:
back_links = {}
for doc in data:
    url_rank = 0
    url_links = []
    for link in doc['links']:
        url_links.append(link)
    back_links[str(doc['_id'])] = url_links
pprint(back_links)

{'24752893': ['6786765789.0', '76675567'],
 '6786765789.0': ['24752893'],
 '76675567': ['24752893', '6786765789.0']}


In [6]:
item_count = 0
for key, value in back_links.items():
    item_count += len(value)
print(item_count)

5


In [7]:
back_links['76675567'].append(132)
pprint(back_links)

{'24752893': ['6786765789.0', '76675567'],
 '6786765789.0': ['24752893'],
 '76675567': ['24752893', '6786765789.0', 132]}


In [8]:
all_docs = {}
for item in data:
    all_docs[str(item['_id'])] = item['new']
pprint(all_docs)

{'24752893': 1.1290058538953526e-05,
 '6786765789.0': 5.645029269476763e-06,
 '76675567': 1.1290058538953526e-05}


In [9]:
ranks = {}
for doc, links in back_links.items():
    new_rank = 0
    for link in links:
        if link in all_docs.keys():
            new_rank += all_docs[link]
        else:
            continue
    ranks[doc] = new_rank
pprint(ranks)

{'24752893': 1.693508780843029e-05,
 '6786765789.0': 1.1290058538953526e-05,
 '76675567': 1.693508780843029e-05}
